<!-- For each driver/rider (person), put in the address and get the locations (long lat) through the geocode api (or any other api). We create a calculate_distance() function that takes in the long lat of 2 locations and returns the absolute distance for 2 locations.

For each driver, we calculate all riders within an arbitrary radius x. We greedily assign all passengers that can fit in this driver's car sorted by the greatest distance from the destination (church) to the furthest driver. We keep track of the distance traveled per passenger from their origin location to the order of their riders R_1, R_2, ... , R_m where m is the capacity of their car. If the amount of riders for their car is not filled, we reset the radius point to the last passenger picked up and 'shift' this origin point in the direction of the destination. We pick up riders on the way until the capacity m is filled.

We do this for a set of drivers. Each driver has a distance score for their certain car. We sum up the distance scores for all drivers after all riders have been assigned (if possible). 

We do this for all permutations of drivers. We use the assignment that minimizes the summed distance score for the drivers.
 -->

(I was brain storming with chat and we agreed this is wayy better)

🧠 Algorithm: Greedy Assignment by Farthest Riders First

Goal:

Assign all riders to drivers such that:
	•	Every rider is picked up.
	•	Each driver can take at most m riders (capacity).
	•	We minimize total driving cost (distance detours).

⸻

🚗 Step-by-Step Explanation:
	1.	Compute distances:
For every rider–driver pair, calculate:
\Delta_{r,d} = \text{dist}(d, r) + \text{dist}(r, \text{destination}) - \text{dist}(d, \text{destination})
This is the added distance for a driver to pick up this rider.
	2.	Sort riders:
Sort all riders in decreasing order of their distance from the destination. This ensures we prioritize assigning the riders who are furthest (and thus hardest to match) first.
	3.	Assign each rider:
For each rider in order:
	•	Among all drivers who still have seats left, find the one with the lowest marginal cost (\Delta_{r,d}).
	•	Assign the rider to that driver.
	•	Reduce the driver’s capacity by 1.
	4.	Repeat until all riders are assigned or no valid assignment exists (e.g., not enough total seats).

In [244]:
import pandas as pd
from collections import defaultdict
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time



In [245]:
df = pd.read_csv('form.csv')

In [246]:
class Driver:
    def __init__(self, name, amount_seats, pickup_location, service_type, plans):
        self.name = name
        self.amount_seats = amount_seats
        self.pickup_location = pickup_location
        self.service_type = service_type
        self.plans = plans
        self.long_lat_pair = ()

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return isinstance(other, Rider) and self.name == other.name

class Rider:
    def __init__(self, name, pickup_location, service_type, plans):
        self.name = name
        self.pickup_location = pickup_location
        self.service_type = service_type
        self.plans = plans
        self.long_lat_pair = ()

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return isinstance(other, Rider) and self.name == other.name

In [247]:
# Dictionary to cache distances between two coordinates.
coords_dict = dict()

def dist(coord1, coord2):
    if (coord1, coord2) in coords_dict:
        return coords_dict[(coord1, coord2)]

    distance_miles = geodesic(coord1, coord2).miles

    coords_dict[(coord1, coord2)] = distance_miles
    coords_dict[(coord2, coord1)] = distance_miles
    return distance_miles  # or .km for kilometers

def assign_riders_by_furthest_first(drivers, riders, destination):
    # Step 1: Precompute distances from riders to destination
    rider_dist_to_dest = {r.name: dist(r.long_lat_pair, destination) for r in riders}

    # Step 2: Sort riders by distance to destination (descending)
    sorted_riders = sorted(riders, key=lambda r: rider_dist_to_dest[r.name], reverse=True)
    # unassigned_riders = set()

    assignments = defaultdict(list)

    for rider in sorted_riders:
        best_driver = None
        min_marginal_cost = float('inf')

        for driver in drivers:
            if driver.amount_seats <= 0:
                continue
            
            # Compute marginal cost of picking this rider
            detour = (
                dist(driver.long_lat_pair, rider.long_lat_pair)
                + dist(rider.long_lat_pair, destination)
                - dist(driver.long_lat_pair, destination)
            )

            if detour < min_marginal_cost:
                best_driver = driver
                min_marginal_cost = detour

        if best_driver is not None:
            assignments[best_driver].append(rider)
            best_driver.amount_seats -= 1
        else:
            print(f"Rider {rider.name} could not be assigned!")
            # unassigned_riders.add(rider)

    return (assignments, unassigned_riders)

In [248]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="ride_assignment_NLF")

# Cache for address → (lat, lon)
address_coords = {}

def geocode_address(address):
    print("GEOCODE_ADDRESS", "address:", address)
    if address in address_coords:
        
        return address_coords[address]
    try:
        location = geolocator.geocode(address)
        if location:
            coord = (location.latitude, location.longitude)
            address_coords[address] = coord
            time.sleep(.01)  # Be respectful to the API (1 req/sec)
            return coord
    except Exception as e:
        print(f"Geocoding failed for '{address}': {e}")
    return None

In [249]:
# Maximum number of distinct locations a driver shoud drive to pick up their passengers
MAXIMUM_LOCATION_THRESHOLD = 1

# Default number of passengers the driver's car can hold.
PASSENGER_LIMIT = 4


In [250]:
people_list = df["Name"]
locations_list = df["Where would you like to be picked up?"]
service_list = df["Which service are you attending?"]
is_driver_list = df["Are you a driver?"]
plans_list = df["Preferred after church plans?"]
oc_address_list = df["Off Campus address"]

drivers = set()
riders = set()

for i in range(len(people_list)):
    val = is_driver_list[i]

    if pd.isna(val) or str(val).strip() == "":
        riders.add(Rider(people_list[i], locations_list[i], service_list[i], plans_list[i]))
    else:
        drivers.add(Driver(people_list[i], PASSENGER_LIMIT, locations_list[i], service_list[i], plans_list[i]))

print("List of all drivers:\n")
for d in drivers:
    print(d.name, d.pickup_location)

print(oc_address_list)

# print("\nList of all riders:\n")
# for r in riders:
#     print(r.name, r.pickup_location)

List of all drivers:

Olivia Chang Life tower
Jonathan Mak North (Brown, Duncan, Jones, Martel, McMurtry)
AZ Ellis South (Baker, Hanszen, Lovett, Sid Richardson, Wiess, Will Rice)
Oriana Tang Off Campus (indicate location in the next question)
Rebecca Lu Off Campus (indicate location in the next question)
Matthew Ahn Life tower
david kim Off Campus (indicate location in the next question)
Josh Paik Off Campus (indicate location in the next question)
Ric Chang Off Campus (indicate location in the next question)
Jack Havemann Life tower
Jenny Sohn North (Brown, Duncan, Jones, Martel, McMurtry)
Brian Seo South (Baker, Hanszen, Lovett, Sid Richardson, Wiess, Will Rice)
Clay Murphy Life tower
Bless Hwang  Off Campus (indicate location in the next question)
Daniel Choi Off Campus (indicate location in the next question)
Jocelyn Lee Off Campus (indicate location in the next question)
Lia kim Off Campus (indicate location in the next question)
0                      NaN
1            7550 Kirby

In [ ]:
import pandas as pd

NORTH_STOP_NAME = "North (Brown, Duncan, Jones, Martel, McMurtry)"
SOUTH_STOP_NAME = "South (Baker, Hanszen, Lovett, Sid Richardson, Wiess, Will Rice)"
LIFETOWER_STOP_NAME = "Life tower"

location_to_address = {
    NORTH_STOP_NAME: "1601 Rice Boulevard, Houston, TX 77005",
    SOUTH_STOP_NAME: "6320 Main St, Houston, TX 77005",
    LIFETOWER_STOP_NAME: "6919 Main St, Houston, TX 77030",
}

drivers = set()
riders = set()
address_to_coord = dict()

oc_people_w_invalid_address = set()

for i in range(len(df)):
    name = df["Name"][i]
    pickup_location = df["Where would you like to be picked up?"][i]
    service_type = df["Which service are you attending?"][i]
    plans = df["Preferred after church plans?"][i]
    is_driver_val = df["Are you a driver?"][i]
    oc_address = df["Off Campus address"][i]

    if pickup_location not in location_to_address and isinstance(oc_address, float):
        print(f"{name} has an empty address cell on the form. They are wanting to pick up from off campus, but they did not specify an address")
        oc_people_w_invalid_address.add(name)

    # Get address based on pickup location
    if pickup_location in location_to_address:
        address = location_to_address[pickup_location]
    else:
        address = oc_address

    # Get or fetch long-lat coordinate
    if address in address_to_coord:
        coord = address_to_coord[address]
        print("address", address, coord)
    else:
        print("sending geocode address for:", name, address)
        coord = geocode_address(address)
        address_to_coord[address] = coord
        address_to_coord[address] = coord

    if pd.isna(is_driver_val) or str(is_driver_val).strip() == "":
        rider = Rider(name, pickup_location, service_type, plans)
        rider.long_lat_pair = coord
        riders.add(rider)
    else:
        driver = Driver(name, PASSENGER_LIMIT, pickup_location, service_type, plans)
        driver.long_lat_pair = coord
        drivers.add(driver)

print(address_to_coord)

sending geocode address for: Kyle Hwang 6320 Main St, Houston, TX 77005
GEOCODE_ADDRESS address: 6320 Main St, Houston, TX 77005
sending geocode address for: Seojin Kwon 7550 Kirby Dr
GEOCODE_ADDRESS address: 7550 Kirby Dr
sending geocode address for: helena song 1601 Rice Boulevard, Houston, TX 77005
GEOCODE_ADDRESS address: 1601 Rice Boulevard, Houston, TX 77005
sending geocode address for: Clay Murphy 6919 Main St, Houston, TX 77030
GEOCODE_ADDRESS address: 6919 Main St, Houston, TX 77030
address 6919 Main St, Houston, TX 77030 (29.7059138, -95.4052109)
address 6320 Main St, Houston, TX 77005 (29.7171081, -95.3993378)
Rebecca Lu has an empty address cell on the form. They are wanting to pick up from off campus, but they did not specify an address
sending geocode address for: Rebecca Lu nan
GEOCODE_ADDRESS address: nan
address 6320 Main St, Houston, TX 77005 (29.7171081, -95.3993378)
sending geocode address for: Jane Yoo 1963 Dryden Rd
GEOCODE_ADDRESS address: 1963 Dryden Rd
address 

In [252]:
assignments, unassigned_riders = (assign_riders_by_furthest_first(drivers, riders, destination=(29.892500, -95.525675)))

def print_assignment(assignments): # driver to list of rider object
    for driver, riders in assignments.items():
        print(f"\nDriver: {driver.name} {driver.pickup_location.split()[0]}")
        for rider in riders:
            print(rider.name, rider.pickup_location.split()[0])

print_assignment(assignments)


Driver: Ric Chang Off
Pedro Flores-Teran Off
Darius Ajebon  North
Daniel Kim  North
Jocelyn Youn North

Driver: Oriana Tang Off
Kaitlyn Kim Off
Faith Chen Off

Driver: Jocelyn Lee Off
Shayla Nguyen Off
Seojin Kwon Off
Khang Le Off
April Tong Off

Driver: Olivia Chang Life
Ethan Yu Life
JJ Lee Life
Nathanael Wang Life
Sehyun Jung Life

Driver: Matthew Ahn Life
Christina Ko Life
Joanna Wei Life
Melody Hong Life
Grace Park Life

Driver: Jack Havemann Life
Sam Ko Off
Jane Yoo Off
Austin Lee Off
Hyeongjun Son South

Driver: AZ Ellis South
Maya Habraken  South
Taeho Choe South
Benjamin Kim South
Grace Sowon Park  South

Driver: Brian Seo South
Kyle Hwang South
Rachel Kim South
Hannah Kim South
Jeffery Huang South

Driver: Clay Murphy Life
derek liang  South
Daniel Kuo South
Grace Kwon South
Jiwang Lee South

Driver: Jonathan Mak North
Elie Park South
Israel Haile South
Hannah Zhang South
Ella Lu South

Driver: Jenny Sohn North
Joann Jung South
Aaron duong South
helena song North
Daniel Song

In [253]:
location_colors = {
    "North": "FFd9ead3",     # light green 3
    "South": "FF93CCEA",     # Light Cornflower Blue 3 
    "Off": "FFFFFFED",       # light yellow
    "Life": "fff4cccc",      # green
    "Back home 💙": "FFD9D2E9",      # light purple
    "RJM": "FFEAD1DC",       # light pink
    "Lunch 💛": "FFFCE5CD",     # light orange
    "Flexible 💚": "FFCFE2F3",        # light blue
    "Refreshments": "FFB6D7A8",
    "NLK 🧡": "FFD9D9D9",

}

In [254]:
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter

def export_assignments_to_excel(rides_to, unassigned_riders_to, rides_from, unassigned_riders_from, output_filename="ride_assignments.xlsx"):
    wb = Workbook()
    ws = wb.active
    ws.title = "Ride Assignments"

    def place_assignments(assignments, unassigned_riders, start_col, key_col, sort_key_driver, sort_key_rider,
                          driver_color_key, rider_color_key, label_plan=False, include_rider_keys_in_legend=True):
        col = start_col
        row = 2
        curr_car_count = 1
        max_passengers = 0
        local_used_cols = set()
        used_keys = set()

        sorted_assignments = sorted(assignments.items(), key=lambda item: sort_key_driver(item[0]))

        for driver, riders in sorted_assignments:
            driver_text = f"Driver: {driver.name}"
            driver_cell = ws.cell(row=row, column=col, value=driver_text)

            driver_key = driver_color_key(driver)
            used_keys.add(driver_key)
            fill_color = location_colors.get(driver_key, "FFFFFF")
            driver_cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
            driver_cell.alignment = Alignment(horizontal="center")
            driver_cell.font = Font(bold=True, underline="single")
            local_used_cols.add(col)

            max_passengers = max(max_passengers, len(riders))

            sorted_riders = sorted(riders, key=sort_key_rider)
            for i, rider in enumerate(sorted_riders):
                rider_text = f"{rider.name}"
                rider_cell = ws.cell(row=row + 1 + i, column=col, value=rider_text)
                rider_key = rider_color_key(rider)
                if include_rider_keys_in_legend:
                    used_keys.add(rider_key)
                fill_color = location_colors.get(rider_key, "FFFFFF")
                rider_cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
                local_used_cols.add(col)

            if curr_car_count % 5 == 0:
                col = start_col
                row += min(8, max_passengers + 3)
            else:
                col += 1
            curr_car_count += 1

        if unassigned_riders:
            col += 1
            driver_cell = ws.cell(row=row, column=col, value="UNASSIGNED DRIVERS")
            driver_cell.fill = PatternFill(start_color="FFCCCCCC", end_color="FFCCCCCC", fill_type="solid")
            driver_cell.alignment = Alignment(horizontal="center")
            driver_cell.font = Font(bold=True, underline="single")
            local_used_cols.add(col)

            sorted_unassigned = sorted(unassigned_riders, key=sort_key_rider)
            for i, rider in enumerate(sorted_unassigned):
                rider_text = f"{rider.name} ({rider.plans})" if label_plan else f"{rider.name}"
                rider_cell = ws.cell(row=row + 1 + i, column=col, value=rider_text)
                rider_key = rider_color_key(rider)
                if include_rider_keys_in_legend:
                    used_keys.add(rider_key)
                fill_color = location_colors.get(rider_key, "FFFFFF")
                rider_cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
                local_used_cols.add(col)

        # Create key
        key_row = 2
        for key in sorted(used_keys):
            key_cell = ws.cell(row=key_row, column=key_col, value=key)
            fill_color = location_colors.get(key, "FFFFFF")
            key_cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
            key_row += 1

        return local_used_cols

    # Define sort and color strategies
    sort_by_pickup = lambda obj: obj.pickup_location.split()[0]
    sort_by_plans = lambda obj: obj.plans
    color_by_pickup = lambda obj: obj.pickup_location.split()[0]
    color_by_plans = lambda obj: obj.plans

    # rides_to (left): include rider pickup locations in key
    used_to = place_assignments(
        rides_to,
        unassigned_riders_to,
        start_col=3,
        key_col=2,
        sort_key_driver=sort_by_pickup,
        sort_key_rider=sort_by_pickup,
        driver_color_key=color_by_pickup,
        rider_color_key=color_by_pickup,
        label_plan=False,
        include_rider_keys_in_legend=True
    )

    # rides_from (right): do NOT include rider pickup in key
    used_from = place_assignments(
        rides_from,
        unassigned_riders_from,
        start_col=11,
        key_col=10,
        sort_key_driver=sort_by_plans,
        sort_key_rider=sort_by_plans,
        driver_color_key=color_by_plans,
        rider_color_key=color_by_pickup,
        label_plan=True,
        include_rider_keys_in_legend=False
    )

    for col_index in used_from.union(used_to).union({2, 10}):
        col_letter = get_column_letter(col_index)
        max_length = 0
        for row_cells in ws.iter_rows(min_col=col_index, max_col=col_index):
            for cell in row_cells:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
        ws.column_dimensions[col_letter].width = max_length + 1

    wb.save(output_filename)
    print(f"Excel file saved to {output_filename}")

In [255]:
export_assignments_to_excel(
    rides_to=assignments,
    unassigned_riders_to=unassigned_riders,
    rides_from=assignments,
    unassigned_riders_from=unassigned_riders
)

Excel file saved to ride_assignments.xlsx
